In [1]:
# !pip install PytorchCML

In [1]:
import sys
sys.path.append("../../src/")

from itertools import product

from PytorchCML import losses, models, samplers, evaluators, trainers
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [2]:
def svd_init(X, dim):
    """
    Args :
        X : csr_matrix which element is 0 or 1.
        dim : number of dimention
    """
    svd = TruncatedSVD(n_components=10)
    U_ = svd.fit_transform(X)
    V_ = svd.components_

    s = (U_.sum(axis=1).mean() + V_.sum(axis=0).mean()) / 2
    U = 2 ** 0.5 * U_ - (1 / n_dim) ** 0.5 * s * np.ones_like(U_)
    V = 2 ** 0.5 * V_ + (1 / n_dim) ** 0.5 / s * np.ones_like(V_)
    ub = -(2 / n_dim) ** 0.5 * U_.sum(axis=1) / s
    vb = (2 / n_dim) ** 0.5 * V_.sum(axis=0) * s

    return U, V, ub, vb

In [3]:
movielens = pd.read_csv(
  'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
  sep='\t', header=None, index_col=None,
)
movielens.columns = ["user_id", "item_id", "rating", "timestamp"]
movielens.user_id -= 1
movielens.item_id -= 1
movielens.rating = (movielens.rating >= 4).astype(int)
n_user = movielens.user_id.nunique()
n_item = movielens.item_id.nunique()

train, test = train_test_split(movielens.copy())


# all user item pairs
df_all = pd.DataFrame(
    [[u, i] for u,i in product(range(n_user), range(n_item))],
    columns=["user_id", "item_id"]
)

# frag train pairs
df_all = pd.merge(
    df_all, 
    train[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
)

# remove train pairs
test = pd.merge(
    df_all[df_all.rating.isna()][["user_id", "item_id"]], 
    test[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
).fillna(0)

# numpy array
train_set = train[train.rating == 1][["user_id", "item_id"]].values
test_set = test[["user_id", "item_id", "rating"]].values

# to torch.Tensor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)


In [4]:
n_dim = 10
X = csr_matrix(
    (np.ones(train_set.shape[0]), (train_set[:,0], train_set[:,1])),
    shape=[n_user, n_item]
)
U, V, ub, vb = svd_init(X, n_dim)

# Naive MF

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 1e-3
n_dim = 10
model = models.LogitMatrixFactorization(
    n_user, n_item, n_dim, max_norm=5,max_bias=3,
    user_embedding_init = torch.Tensor(U), 
    item_embedding_init = torch.Tensor(V.T),
    user_bias_init = torch.Tensor(ub), 
    item_bias_init = torch.Tensor(vb)
).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.LogitPairwiseLoss().to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device,n_neg_samples=5, batch_size=1024)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3])
trainer = trainers.MFTrainer(model, optimizer, criterion, sampler)


In [6]:
trainer.fit(n_batch=50, n_epoch=15, valid_evaluator = evaluator, valid_per_epoch=5)

100%|██████████| 943/943 [00:20<00:00, 46.42it/s]


In [7]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,epoch,loss
0,0.404477,0.545157,0.111933,0,NaN
0,0.408631,0.549399,0.115896,5,0.356066
0,0.399501,0.540827,0.115340,10,0.308907
0,0.395533,0.534553,0.116172,15,0.289745


# RelMF

In [8]:
train["popularity"] = train.groupby("item_id").rating.transform(sum)
train["pscore"] = 1 / (train.popularity / train.popularity.max()) ** 0.5

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 1e-3
n_dim = 10

train_set = train[train.rating == 1][["user_id", "item_id", "pscore"]].values
train_set = torch.LongTensor(train_set).to(device)

model = models.LogitMatrixFactorization(
    n_user, n_item, n_dim, max_norm=5,max_bias=3,
    user_embedding_init = torch.Tensor(U), 
    item_embedding_init = torch.Tensor(V.T),
    user_bias_init = torch.Tensor(ub), 
    item_bias_init = torch.Tensor(vb)
).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.RelevancePairwiseLoss(delta="rmse").to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device,n_neg_samples=5, batch_size=1024)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3])
trainer = trainers.MFTrainer(
    model, optimizer, criterion, sampler, 
    column_names={"user_id":0, "item_id":1, "pscore":2}
)


In [13]:
trainer.fit(n_batch=50, n_epoch=15, valid_evaluator = evaluator, valid_per_epoch=5)

100%|██████████| 943/943 [00:18<00:00, 50.99it/s]


In [14]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,epoch,loss
0,0.404477,0.545157,0.111933,0,NaN
0,0.398088,0.544185,0.112874,5,0.308484
0,0.392882,0.537027,0.112438,10,0.186894
0,0.393925,0.533581,0.112751,15,0.145944
